## Preprocessing and Analysis of snRNAseq Data

In this notebook I do quality control, preprocessing, and data analysis of single nuclei sequencing data (gene count matrices) generated by the Smart-seq3xpress protocol. For this I am mainly using the ScanPy module.<br>
**Data used here is of the Spiral Ganglion Aging Atlas**

The workflow presented here contains:
- Reading the data
- Removing annotation layers and only reatining some selected annotations
- Quality control (determining outliers by counts as well as mitochondrial genes)
- Normalization using analytic Pearson residuals
- Feature selection
- Dimenstionality reduction and clustering
- UMAP embedding
- Manual cell type annotation
- Visualization of marker expression

Some quick features:
- We filter out genes that occur in <3 cells
- We filter out cells that have < 2000 genes
- We filter out cells that are outside 4 median absolute deviations of distinct gene counts or total gene counts
- We filter out mitochondrial genes
- Batch covariate is Age
- We normalize using analytic Pearson residuals

In [1]:
# Importing necessary modules for ...
# ... data processing ...
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import scipy.stats as st
import bbknn
ll
# ... plotting ...
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc_context
from matplotlib.colors import ListedColormap
import matplotlib.font_manager as fm
import seaborn as sns

# ... reading and writing files
import sys, os
import json

C:\Users\Johann\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Plotting Configuration

This cell is simply for making pretty plots.

In [ ]:
# Defining custom color map for plots
with open(r".\Resources\ColorMap.csv", "r") as file:
    my_cmap_df = pd.read_csv(file, header=0)
    my_cmap = ListedColormap(my_cmap_df["HEX_Code"].to_list())

# Defining save location for plots
figures_folder = r".\Plots\\"

# Defining folder where raw data is stored
data_folder = r".\Data\\"

# Adding custom font
fonts= [r"Resources\Arimo\Arimo-Regular.ttf",
        r".\Resources\Arimo\Arimo-SemiBold.ttf",
        r".\Resources\Arimo\Arimo-Italic.ttf",
        r".\Resources\Arimo\Arimo-SemiBoldItalic.ttf"]
for font in fonts:
    fm.fontManager.addfont(font)

# Setting parameters for ScanPy plots
sc.set_figure_params(
    dpi= 150,
    dpi_save= 450,
    facecolor= "white",
    color_map= "RdPu")

scale = 1.4 #plot scale factor
fig_width = 2.35*scale

plt.rcParams["font.family"] = "Arimo"
plt.rcParams["figure.facecolor"] = (1.0, 1.0, 1.0, 0.0)
plt.rcParams["axes.labelsize"] = 6*scale
plt.rcParams["axes.labelweight"] = "semibold"
plt.rcParams["axes.facecolor"] = (1.0, 1.0, 1.0, 0.7)
plt.rcParams["xtick.labelsize"] = 6*scale
plt.rcParams["ytick.labelsize"] = 6*scale
plt.rcParams["axes.titlesize"] = 8*scale
plt.rcParams["axes.titlelocation"] = "left"
plt.rcParams["axes.titleweight"] = "semibold"
plt.rcParams["legend.fontsize"] = 6*scale

### Reading Data

We read the data from an .h5ad file that contains the raw count matrix with some annotations and metadata. Between all data sets we calculate the intersection of features contained (common genes between them) and limit each data set to only the genes that are present in all of them.

Information carried over is:
- plate ID (a unique barcode for each 384-well plate sequenced)
- age of animals

In [ ]:
# Reading data
adatas = {"adata_3mo":  sc.read_h5ad(data_folder + r"CBA_3mo_SGN.h5ad"),
          "adata_12mo": sc.read_h5ad(data_folder + r"CBA_12mo_SGN.h5ad"),
          "adata_24mo": sc.read_h5ad(data_folder + r"CBA_24mo_SGN.h5ad")}

# Getting list of genes in all data sets
features_3mo = adatas["adata_3mo"].var_names.to_list()
features_12mo = adatas["adata_12mo"].var_names.to_list()
features_24mo = adatas["adata_24mo"].var_names.to_list()
print(f"Identified {len(features_3mo)} genes in first data set.")
print(f"Identified {len(features_12mo)} genes in second data set.")
print(f"Identified {len(features_24mo)} genes in third data set.\n")

common_features = np.intersect1d(np.intersect1d(features_3mo, features_12mo), features_24mo).tolist()
print(f"Identified {len(common_features)} common genes between data sets.", common_features, sep= "\n")

adatas["adata_3mo"] = adatas["adata_3mo"][:, adatas["adata_3mo"].var_names.isin(common_features)].copy()
adatas["adata_12mo"] = adatas["adata_12mo"][:, adatas["adata_12mo"].var_names.isin(common_features)].copy()
adatas["adata_24mo"] = adatas["adata_24mo"][:, adatas["adata_24mo"].var_names.isin(common_features)].copy()

print(f"\nFiltered dataset 1 has {adatas["adata_3mo"].n_obs} nuclei and {adatas["adata_3mo"].n_vars} genes.\n\
Filtered dataset 2 has {adatas["adata_12mo"].n_obs} nuclei and {adatas["adata_12mo"].n_vars} genes.\n\
Filtered dataset 3 has {adatas["adata_24mo"].n_obs} nuclei and {adatas["adata_24mo"].n_vars} genes.")

# Concatenating all age groups
adata_original = ad.concat([adatas["adata_3mo"], adatas["adata_12mo"], adatas["adata_24mo"]],
                           label= "age",
                           keys= ["3 months", "12 months", "24 months"])

# Making a clean copy ...
adata = ad.AnnData(adata_original.X.copy())

# # ... and preserving some of the metadata
adata.obs["plate"] = adata_original.obs["plate"].values.astype(str)
adata.obs["age"] = adata_original.obs["age"].values
adata.obs.set_index(adata_original.obs.index, inplace= True)
adata.var.set_index(adata_original.var.index, inplace= True)

adata

### Quality Control

We calculate quality control metrics. These include:
- total gene counts per nucleus
- distinct genes per nucleus
- percentages of the defined gene sets (mito, ribo, hemo)

In [ ]:
# Define mitochondrial, ribosomal, and hemoglobin genes by name
# (they are already filtered out in this dataset ...)
adata.var["mt"] = adata.var_names.str.startswith("mt-")
adata.var["ribo"] = adata.var_names.str.startswith(("Rpl", "Rps"))
adata.var["hb"] = adata.var_names.str.startswith("Hb")

print(f"{len(adata.var[adata.var["mt"]])} Mitochondrial genes identified:\n", adata.var_names[adata.var["mt"]].values, "\n")
print(f"{len(adata.var[adata.var["ribo"]])} Ribosomal genes identified:\n", adata.var_names[adata.var["ribo"]].values, "\n")
print(f"{len(adata.var[adata.var["hb"]])} Hemoglobin genes identified:\n", adata.var_names[adata.var["hb"]].values)

# Calculating QC metrics including the gene subsets
sc.pp.calculate_qc_metrics(adata, qc_vars= ["mt", "ribo", "hb"],
                           inplace= True,
                           log1p= True)

adata.obs

### Calling Outliers

We define outliers nuclei as follows:
- total counts outside of 4 median absolute deviations (MADs) OR
- distinct counts outside of 4 MADs OR
- distinct counts < 2000 genes

The latter is a manual hard cut-off that is based on the minimum performance expected from Smart-seq3xpress.

In [ ]:
# Calculating MAD for a given QC metric
def is_outlier(adata, metric: str, nmads: int):
    '''This function calculates whether each element of a list or series is an
    outlier based on the number of median absolute deviations (MADs) passed in the
    arguments nmads. It returns a list of booleans where each element that was an outlier
    in the original list results in a TRUE in the returned list. If you use the returned
    list for indexing to EXCLUDE outliers, remember to index with the inverse (~outliers) of the outlier
    list returned by this function'''
    M = adata.obs[metric]
    outlier = ((M < np.median(M) - nmads * st.median_abs_deviation(M)) # outlier below ...
               | (np.median(M) + nmads * st.median_abs_deviation(M) < M)) # and outlier above
    return outlier

# Defining cut-off for mitochondrial percentage
max_mito_pct = 0.3

# Defining whether a nucleus is an outlier based on total counts and gene counts per cell
adata.obs["outlier"] = (is_outlier(adata, "log1p_total_counts", 4)
                        | is_outlier(adata, "log1p_n_genes_by_counts", 4)
                        | (adata.obs["n_genes_by_counts"] < 2000)) # Custom hard threshold for minimum gene number per nucleus due to Smart-seq3xpress performance

# Running doublet detection and creating a doublet score for every cell
sc.pp.scrublet(adata, batch_key= "age")

print(f"Identified {len(adata.obs[adata.obs["outlier"]])} out of {len(adata.obs)} nuclei as outliers.\n",
      "Nuclei remaining of each age group: \n",
      adata[~adata.obs["outlier"]].obs["age"].value_counts())

In [ ]:
# Plotting Quality Control metrics
fig, axes = plt.subplots(3, 4, figsize=(fig_width*4.2, fig_width*3), layout= "constrained")

age_colors = [my_cmap.colors[4],
              my_cmap.colors[6],
              my_cmap.colors[5]]
# Gene per Nuclei (all samples)
sc.pl.violin(adata, ["n_genes_by_counts"], jitter= 0.4, ax= axes[1, 0], show= False, palette = my_cmap.colors)
axes[1, 0].set_xticklabels(["Single Nuclei"])
axes[1, 0].set_ylabel("Number of Distinct Genes")
axes[1, 0].axhline(adata[~adata.obs["outlier"]].obs["n_genes_by_counts"].min(),
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[1, 0].axhline(adata[~adata.obs["outlier"]].obs["n_genes_by_counts"].max(),
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[1, 0].text(-0.4, adata[~adata.obs["outlier"]].obs["n_genes_by_counts"].max()+170, "outlier cutoff",
                color= my_cmap.colors[1],
                fontsize= 6*scale,
                fontstyle= "italic")

# Genes per Nuclei (grouped by age)
sc.pl.violin(adata, ["n_genes_by_counts"], jitter= 0.4, ax= axes[1, 2], show= False, palette = age_colors, groupby= "age")
axes[1, 2].set_xlabel("Single Nuclei")
axes[1, 2].set_ylabel("Number of Distinct Genes")
axes[1, 2].axhline(adata[~adata.obs["outlier"]].obs["n_genes_by_counts"].min(),
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[1, 2].axhline(adata[~adata.obs["outlier"]].obs["n_genes_by_counts"].max(),
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[1, 2].text(-0.4, adata[~adata.obs["outlier"]].obs["n_genes_by_counts"].max()+170, "outlier cutoff",
                color= my_cmap.colors[1],
                fontsize= 6*scale,
                fontstyle= "italic")

# Total Counts (all samples)
sc.pl.violin(adata, ["total_counts"], jitter= 0.4, ax= axes[1, 1], show= False, palette = my_cmap.colors)
axes[1, 1].set_xticklabels(["Single Nuclei"])
axes[1, 1].set_ylabel("Total UMI Counts")
axes[1, 1].axhline(adata[~adata.obs["outlier"]].obs["total_counts"].min(),
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[1, 1].axhline(adata[~adata.obs["outlier"]].obs["total_counts"].max(),
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[1, 1].text(-0.4, adata[~adata.obs["outlier"]].obs["total_counts"].max()+2000, "outlier cutoff",
                color= my_cmap.colors[1],
                fontsize= 6*scale,
                fontstyle= "italic")
axes[1, 1].set_ylim(0, 100000)
axes[1, 1].text(-0.4, 97000, "y-axis cut for visibility",
                color= "black",
                fontsize= 4*scale,
                fontstyle= "italic")

# Total Counts (grouped by age)
sc.pl.violin(adata, ["total_counts"], jitter= 0.4, ax= axes[1, 3], show= False, palette = age_colors, groupby= "age")
axes[1, 3].set_xlabel("Single Nuclei")
axes[1, 3].set_ylabel("Total UMI Counts")
axes[1, 3].axhline(adata[~adata.obs["outlier"]].obs["total_counts"].min(),
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[1, 3].axhline(adata[~adata.obs["outlier"]].obs["total_counts"].max(),
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[1, 3].text(-0.4, adata[~adata.obs["outlier"]].obs["total_counts"].max()+2000, "outlier cutoff",
                color= my_cmap.colors[1],
                fontsize= 6*scale,
                fontstyle= "italic")
axes[1, 3].set_ylim(0, 100000)
axes[1, 3].text(-0.4, 97000, "y-axis cut for visibility",
                color= "black",
                fontsize= 4*scale,
                fontstyle= "italic")

# Mitochondrial percentage (all samples)
sc.pl.violin(adata, ["pct_counts_mt"], jitter= 0.4, ax= axes[2, 0], show= False, palette = my_cmap.colors)
axes[2, 0].set_xticklabels(["Single Nuclei"])
axes[2, 0].set_ylabel("Percentage of Mitochondrial Genes")
axes[2, 0].set_ylim(0, 1)
axes[2, 0].text(-0.4, 0.95, "y-axis cut for visibility",
                color= "black",
                fontsize= 4*scale,
                fontstyle= "italic")
axes[2, 0].axhline(max_mito_pct,
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[2, 0].text(-0.4, max_mito_pct+max_mito_pct*0.1, "outlier cutoff",
                color= my_cmap.colors[1],
                fontsize= 6*scale,
                fontstyle= "italic")

# Mitochondrial percentage (grouped by age)
sc.pl.violin(adata, ["pct_counts_mt"], jitter= 0.4, ax= axes[2, 2], show= False, palette = age_colors, groupby= "age")
axes[2, 2].set_xlabel("Single Nuclei")
axes[2, 2].set_ylabel("Percentage of Mitochondrial Genes")
axes[2, 2].set_ylim(0, 1)
axes[2, 2].text(-0.4, 0.95, "y-axis cut for visibility",
                color= "black",
                fontsize= 4*scale,
                fontstyle= "italic")
axes[2, 2].axhline(max_mito_pct,
                   color= my_cmap.colors[1],
                   linestyle= "--",
                   zorder= 3)
axes[2, 2].text(-0.4, max_mito_pct+max_mito_pct*0.1, "outlier cutoff",
                color= my_cmap.colors[1],
                fontsize= 6*scale,
                fontstyle= "italic")

# Ribosomal percentage (all samples)
sc.pl.violin(adata, ["pct_counts_ribo"], jitter= 0.4, ax= axes[2, 1], show= False, palette = my_cmap.colors)
axes[2, 1].set_xticklabels(["Single Nuclei"])
axes[2, 1].set_ylabel("Percentages of Ribosomal Genes")
axes[2, 1].set_ylim(0, 5)
axes[2, 1].text(-0.4, 4.8, "y-axis cut for visibility",
                color= "black",
                fontsize= 4*scale,
                fontstyle= "italic")

# Ribosomal percentage (grouped by age)
sc.pl.violin(adata, ["pct_counts_ribo"], jitter= 0.4, ax= axes[2, 3], show= False, palette = age_colors, groupby= "age")
axes[2, 3].set_xlabel("Single Nuclei")
axes[2, 3].set_ylabel("Percentages of Ribosomal Genes")
axes[2, 3].set_ylim(0, 5)
axes[2, 3].text(-0.4, 4.8, "y-axis cut for visibility",
                color= "black",
                fontsize= 4*scale,
                fontstyle= "italic")

# Highest expressed genes (all samples)
sc.pl.highest_expr_genes(adata, n_top=20, ax= axes[0, 1], show= False, palette= my_cmap.colors)
axes[0, 1].set_title("Highest Expressed Genes Total")
axes[0, 1].set_xlabel(r"% of Total Counts")
axes[0, 1].set_xlim(0, 10)
axes[0, 1].text(9.6, 19, "x-axis cut for visibility",
                color= "black",
                fontsize= 4*scale,
                rotation= 90,
                fontstyle= "italic")

# Total Count Distribution (all samples)
sns.histplot(adata.obs["total_counts"], bins = 100, kde= False, ax= axes[0, 0], color= my_cmap.colors[0])
axes[0, 0].set_title("Distribution of Total Counts")
axes[0, 0].set_xlabel("Total UMI Counts")
axes[0, 0].set_ylabel("Number of Nuclei")
axes[0, 0].set_xlim(0, 100000)
axes[0, 0].text(95000, 150, "x-axis cut for visibility",
                color= "black",
                fontsize= 4*scale,
                rotation= 90,
                fontstyle= "italic")

# Total Counts Distribution (grouped by age)
sns.histplot(adata.obs, x= "total_counts", bins = 100, kde= False, ax= axes[0, 2], palette= age_colors, hue= "age", hue_order= ("3 months", "12 months", "24 months"))
axes[0, 2].set_title("Distribution of Total Counts")
axes[0, 2].set_xlabel("Total UMI Counts")
axes[0, 2].set_ylabel("Number of Nuclei")
axes[0, 2].legend(["3 months", "12 months", "24 months"], title= "Age Group", title_fontsize= 6*scale, frameon= False)
axes[0, 2].set_xlim(0, 100000)
axes[0, 2].text(95000, 150, "x-axis cut for visibility",
                color= "black",
                fontsize= 4*scale,
                rotation= 90,
                fontstyle= "italic")

fig.tight_layout(h_pad= 1.0*scale)
fig.savefig(figures_folder + r"QC_Metrics_SGNAtlas.png")
plt.show()

### Normalization

We normalize counts using analytical pearson residuals.

In [ ]:
# Saving a copy of the raw counts before normalising
adata.layers["raw_counts"] = adata.X.copy()

In [ ]:
# Normalize the data and store the normalizations in separate layers ...
# ... by pearson residulals ...
adata.layers["pearson_normalized_counts"] = sc.experimental.pp.normalize_pearson_residuals(adata, copy= True).X

# ... and by median (with log1p transformation)
adata.layers["log1p_normalized_counts"] = sc.pp.normalize_total(adata, copy= True).X
sc.pp.log1p(adata, layer= "log1p_normalized_counts")

fig, axes = plt.subplots(1, 3, figsize=(fig_width*2.4, fig_width*0.8), layout= "constrained")

# the original layer remains raw
sns.histplot(adata.X.sum(1), bins = 100, kde= False, ax= axes[0], palette= my_cmap.colors, legend= False)
axes[0].set_title("Non-normalized")

sns.histplot(adata.layers["pearson_normalized_counts"].sum(1), bins = 100, kde= False, ax= axes[1], color= my_cmap.colors[0])
axes[1].set_title("Analytical Pearson Residuals")


sns.histplot(adata.layers["log1p_normalized_counts"].sum(1), bins = 100, kde= False, ax= axes[2], palette= my_cmap.colors, legend= False)
axes[2].set_title("Median + Log1p")

plt.show()
fig.savefig(figures_folder + r"Normalization_SGNAtlas.png")

### Feature Selection

We select the top 2000 most highly variable genes (HVGs) based on pearson normalized and median normalized data and save the annotation so separate fields.

**DO NOT RUN THE FOLLOWING CELL TWICE WITHOUT RELOADING THE DATA OR RESTARTING THE KERNEL**

In [ ]:
# Feature selection based on pearson residuals (uses the raw counts by default) ...
sc.experimental.pp.highly_variable_genes(adata, n_top_genes= 2000, batch_key= "age")

# ... and based on log1p normalized counts
if "log1p_highly_variable" in adata.obs_names.values:
    pass
else:
    adata.var = pd.concat((adata.var,
                        sc.pp.highly_variable_genes(adata, n_top_genes= 2000,
                                                    layer= "log1p_normalized_counts",
                                                    inplace= False).rename(
                                                        columns={"highly_variable": "log1p_highly_variable",
                                                                    "means": "log1p_means",
                                                                    "dispersions": "log1p_dispersions",
                                                                    "dispersions_norm": "log1p_dispersions_norm"}).loc[:, ["log1p_highly_variable", "log1p_means", "log1p_dispersions", "log1p_dispersions_norm"]]), axis=1)

In [ ]:
# Plotting highly variable gene fraction
fig, ax = plt.subplots(1,2, figsize=(fig_width*1.6, fig_width*0.8), layout= "constrained")

sns.scatterplot(adata.var, x= "means", y= "residual_variances",
                hue= "highly_variable", 
                palette= my_cmap.colors,
                ax= ax[0], 
                s= 5,
                edgecolor= "black",
                linewidth= 0.1,
                zorder= 2)
ax[0].set_title("Highly Variable Genes Pearson")
ax[0].legend()
ax[0].set_xscale("log")
ax[0].set_yscale("log")

sns.scatterplot(adata.var, x= "log1p_means", y= "log1p_dispersions",
                hue= "log1p_highly_variable",
                palette= my_cmap.colors,
                ax= ax[1],
                s= 5,
                edgecolor= "black",
                linewidth= 0.1,
                zorder= 2)
ax[1].set_title("Highly Variable Genes Log1p")
ax[1].set_xscale("log")
ax[1].legend()

fig.savefig(figures_folder + r"HVGs_SGNAtlas.png")

### Dimensionality Reduction

We compute principal components on the pearson normalized count matrix.

In [ ]:
# Making sure raw counts are used
adata.X = adata.layers["raw_counts"].copy()

# Computing PCA with pearson residuals on the raw count matrix
sc.experimental.pp.normalize_pearson_residuals_pca(adata)

# We inspect the PCA results
fig, axes = plt.subplots(2, 3, figsize=(fig_width*3.5, fig_width*1.6), layout= "compressed")

sc.pl.pca_variance_ratio(adata, show= False)
axes[0, 0].set_title("Variance Ratios")

# PCA by plate
sc.pl.pca(adata, color= "plate",
          palette= my_cmap.colors,
          s= 10,
          edgecolor= "black",
          linewidth= 0.1,
          ax= axes[0, 1],
          sort_order= False,
          show= False)
axes[0, 1].set_title("PCA Results (by Plate ID)")

# PCA by total counts
sc.pl.pca(adata, color= "total_counts",
          s= 10,
          edgecolor= "black",
          linewidth= 0.1,
          ax= axes[0, 2],
          sort_order= False,
          show= False)
axes[0, 2].set_title("PCA Results by Total Counts")

# PCA by age group
sc.pl.pca(adata, color= "age",
          palette= my_cmap.colors,
          s= 10,
          edgecolor= "black",
          linewidth= 0.1,
          ax= axes[1, 0],
          sort_order= False,
          show= False)
axes[1, 0].set_title("PCA Results (by age group)")

fig.savefig(figures_folder + r"PCA_SGNAtlas.png")
plt.show()

### UMAP Embedding

We batch correct the data by calculating a batch-balances k-nearest neighbor (BBKNN) graph and calculate a UMAP projection.
The batch covariate is the plateID, so we correct for differences between each plate.

In [ ]:
# Making sure normalized counts are used
adata.X = adata.layers["pearson_normalized_counts"].copy()

# BBKNN embedding
bbknn.bbknn(adata, batch_key= "age", neighbors_within_batch= 3)

# UMAP Embedding
sc.tl.umap(adata, min_dist= 1.3, spread= 1.6)

In [ ]:
# Plotting before filtering
fig, axs = plt.subplots(2, 3, figsize= (fig_width*3.5, fig_width*1.6), layout= "compressed")

metrics = ["total_counts", "age", "predicted_doublet", "n_genes_by_counts", "age", "plate"]
dot_size = 6
vmax = "p99"
for i, metric in enumerate(metrics):
    if metric == "age":
        cmap = [my_cmap.colors[4],
                my_cmap.colors[6],
                my_cmap.colors[5]]
    else:
        cmap = my_cmap.colors
    sc.pl.umap(adata,
            color=[metric],
            palette= cmap,
            size= dot_size,
            vmax= vmax,
            edgecolor= "black",
            linewidth= 0.1,
            ax= axs[int(i/(len(metrics)/2)), int(i%(len(metrics)/2))],
            show= False)
axs[0, 0].text(-16,27, f"n = {adata.n_obs}", size= 8)

fig.savefig(figures_folder + r"UMAP_Prefilter_SGNAtlas.png")
plt.show()

### Filtering

We filter out cells and genes that we determined to be outliers as defined above.<br>
Exclusion criteria:
- total genes counts outside of 4 MADs in positive or negative direction
- less than 1000 genes expressed per nucleus (due to Smart-seq3xpress performance)
- single genes expressed in less than 3 cells in data set
- all mitochondrial genes

In [ ]:
# And this is also what we have to do with out custom "outlier" annotations
print(f"Number of nuclei before outliers: {adata.n_obs}\n\
Number of genes before outliers: {adata.n_vars}")
adata = adata[(~adata.obs["outlier"]), ~adata.var["mt"]]
print(f"\nNumber of nuclei after outliers: {adata.n_obs}\n\
Number of genes after outliers: {adata.n_vars}")

print(f"\nNumber of nuclei before count exclusion: {len(adata.obs)}")
sc.pp.filter_genes(adata, min_cells= 3, inplace= True)
print(f"Number of nuclei after count exclusion: {len(adata.obs)}")

In [ ]:
# Plotting before filtering
fig, axs = plt.subplots(2, 3, figsize= (fig_width*3.5, fig_width*1.6), layout= "compressed")

metrics = ["total_counts", "age", "predicted_doublet", "n_genes_by_counts", "age", "plate"]
dot_size = 6
vmax = "p99"
for i, metric in enumerate(metrics):
    if metric == "age":
        cmap = [my_cmap.colors[4],
                my_cmap.colors[6],
                my_cmap.colors[5]]
    else:
        cmap = my_cmap.colors
    sc.pl.umap(adata,
            color=[metric],
            palette= cmap,
            size= dot_size,
            vmax= vmax,
            edgecolor= "black",
            linewidth= 0.1,
            ax= axs[int(i/(len(metrics)/2)), int(i%(len(metrics)/2))],
            show= False)
axs[0, 0].text(-16,27, f"n = {adata.n_obs}", size= 8)

fig.savefig(figures_folder + r"UMAP_Postfilter_SGNAtlas.png")
plt.show()

### Clustering and Annotating

We cluster using the Leiden algorithm at an initial resolution of 1.

In [ ]:
adata.X = adata.layers["pearson_normalized_counts"].copy() # The leiden() function requires normalized counts

# Clustering ...
sc.tl.leiden(adata,
             flavor= "igraph",
             n_iterations= -1, # iterate until the best clusters have been found
             resolution= 1, # higher number means finer clusters
             key_added= "leiden")

# Plotting the clusters
fig, ax = plt.subplots(1,1, figsize= (fig_width*0.9, fig_width*0.9))

sc.pl.umap(adata, color= ["leiden"],
           palette= my_cmap.colors,
           size= 6,
           edgecolor= "black",
           linewidth= 0.1,
           legend_loc= "on data",
           legend_fontsize = 3,
           legend_fontoutline = 0.4,
           title= "Leiden Clustering (resolution 1)",
           ax= ax,
           show= False)

fig.savefig(figures_folder + r"Leiden_SGNAtlas.png")
plt.show()

#### Loading Marker Genes

A list of literature markers of different cochlear cell types. The same ones as used for the reference atlas.

In [ ]:
# Marker Genes
# Loading previously saved markers
with open(r"Resources\cochlea_markers.json", "r") as file:
    markers_SGN = json.load(file)

# Checking which marker genes are present in the dataset
markers_in_data = {}
for ct, markers in markers_SGN.items():
    if np.intersect1d(markers, adata.var_names).size > 0:
        markers_in_data[ct] = list(np.intersect1d(markers, adata.var_names))

# printing the final list of available marker genes
print("Marker genes present in data:\n")
for item in markers_in_data.items():
    print(f"{item[0]}: {item[1]}")

### Checking Marker Expression

We visualize the expression of the marker genes in each cluster and infer which clusters could belong to which cell type.

Initally we just observe if the markers locate to clusters unambiguously, if not we sub-cluster the clusters that are too coarse in the next step.

In [ ]:
# Next, we check the expression patterns of the marker genes in our clustering
sc.tl.leiden(adata,
             flavor= "igraph",
             n_iterations= -1,
             resolution= 0.8,
             key_added= "leiden_0")

# Dotplot featuring all marker genes and all the clusters to manually determine cell identities in clusters
# THIS IS WHERE THE ACTUAL CLUSTER ANNOTATION CALLING HAPPENS
sc.pl.dotplot(adata,
              markers_in_data,
              groupby="leiden_0",
              standard_scale="var",
              cmap= "Purples", # I like purple
            #   save= "Marker_Genes_SGNAtlas.png", # save the dotplot, comment out if desired
              )

sc.pl.umap(adata,
           color=["leiden_0"],
           legend_loc="on data",
           legend_fontsize= 2,
           legend_fontoutline= 0.8,
           size= 6,
           palette= my_cmap.colors,
           edgecolor= "black",
           linewidth= 0.1)

In [ ]:
# After initial inspection of clustering, we defining clusters that look like they wanna be clustered further
# This is iterative sub-clustering
need_subclustering = {"1": 0.5,
                      "10": 0.5,
                      "11": 0.2}

# Successive sub-clustering of clusters that are too coarse to tell cell types apart
i = 1
for cluster, res in need_subclustering.items():
    sc.tl.leiden(adata, flavor= "igraph", n_iterations= -1, key_added= f"leiden_{i}",
                 resolution= res,
                 restrict_to= (f"leiden_{i-1}", [cluster]))
    i += 1

# Dotplot featuring all marker genes and all the clusters to manually determine cell identities in clusters
# THIS IS WHERE THE ACTUAL CLUSTER ANNOTATION CALLING HAPPENS
sc.pl.dotplot(adata,
              markers_in_data,
              groupby=f"leiden_{len(need_subclustering)}",
              standard_scale= "var",
              cmap= "Purples", # I like purple
            #   save= "Marker_Genes_SGNAtlas.png", # Comment in if wanted
              )

sc.pl.umap(adata,
           color=[f"leiden_{len(need_subclustering)}"],
           legend_loc="on data",
           legend_fontsize= 2,
           legend_fontoutline= 0.8,
           size= 6,
           palette= my_cmap.colors,
           edgecolor= "black",
           linewidth= 0.1)

### Checking Cluster Expressions

We visualise marker gene expressions on the UMAP projection to visually verify localizations.

In [ ]:
# Defining a subgroup of cell types of which to check expression levels across the whole data set
# Change cell types as needed!
sgns = [
    "Spiral Ganglion Neuron Type Ia",
    "Spiral Ganglion Neuron Type Ib",
    "Spiral Ganglion Neuron Type Ic",
    "Spiral Ganglion Neuron Type II"]

# Now we plot marker gene expression for each marker of the above defined cell types
for ct in sgns:
    print(f"{ct}:")  # print cell subtype name
    sc.pl.umap(adata,
               color= [*markers_in_data[ct], f"leiden_{len(need_subclustering)}"],
               vmax="p99",
               sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cell,
               frameon=False,
               size= 6,
               palette= my_cmap.colors,
               edgecolor= "black",
               linewidth= 0.1)

### Comitting Annotations

We now define a manual cell type for each numeric cluster from before and map the annotated cells to a new field in the .obs annotations (every nucleus gets a cell type label).

In [ ]:
# Committing Annotations
annotations = {"0": "Myelinating Schwann Cells",
               "1,0": "Fibrocytes I",
               "1,1": "Fibrocytes I",
               "1,2": "Fibrocytes II",
               "1,3": "Fibrocytes IV",
               "1,4": "Unclear",
               "1,5": "Unclear",
               "2": "Satellite Glial Cells 1",
               "3": "Myelinating Schwann Cells",
               "4": "Myelinating Schwann Cells",
               "5": "Endothelial Cells",
               "6": "Satellite Glial Cells 2",
               "7": "Fibroblasts",
               "8": "Osteocytes",
               "9": "Chondrocytes",
               "10,0": "Spiral Ganglion Neuron Type Ib",
               "10,2": "Spiral Ganglion Neuron Type Ib",
               "10,1": "Spiral Ganglion Neuron Type Ic",
               "10,3": "Spiral Ganglion Neuron Type Ic",
               "11,0": "Spiral Ganglion Neuron Type II",
               "11,1": "Unclear",
               "11,2": "Unclear",
               "11,3": "Unclear",
               "12": "Fibrocyte III",
               "13": "Spiral Ganglion Neuron Type II",
               "14": "Oligodendrocytes",
               "15": "Spiral Ganglion Neuron Type Ia",
               "16": "Satellite Glial Cells 1",
               "17": "Myelinating Schwann Cells",
               "18": "Macrophages",
               "19": "Non-myelinating Schwann Cells",
               "20": "Unclear",
               "21": "Unclear",
               "22": "Myelinating Schwann Cells",
               "23": "Myelinating Schwann Cells",
               "24": "Myelinating Schwann Cells",
               "25": "Myelinating Schwann Cells",
               "26": "Myelinating Schwann Cells",
               "27": "Myelinating Schwann Cells",
               "28": "Myelinating Schwann Cells"}

# Refactoring the cluster identity and saving it to a new annotation as cell types
adata.obs["manual_celltypes"] = adata.obs[f"leiden_{len(need_subclustering)}"].map(annotations)

# Saving the annotated dataset
adata.write_h5ad(data_folder + r"CBA_SGNAtlas_Annotated.h5ad")

In [ ]:
# Plotting Cell Type Annotations
fig, ax = plt.subplots(2,1, figsize=(fig_width*2.1, fig_width*1.8), layout= "compressed")

sc.pl.umap(adata,
           color=["manual_celltypes"],
           edgecolor= "black",
           linewidth= 0.15,
           size= 10,
           palette= my_cmap.colors,
           legend_fontsize= 6*scale,
           ax = ax[0],
           show= False)
ax[0].set_title("SGN Aging Atlas")

sc.pl.umap(adata,
           color=["age"],
           edgecolor= "black",
           linewidth= 0.15,
           size= 10,
           palette= [my_cmap.colors[4],
                     my_cmap.colors[6],
                     my_cmap.colors[5]],
           legend_fontsize= 6*scale,
           ax = ax[1],
           show= False)
ax[1].set_title("Age Difference", fontsize= 10)

fig.savefig(figures_folder + r"Annotations_SGNAtlas.png")
plt.show()

## Gene Expression Analysis

Here, as an example, we filter out only the spiral ganglion neurons and plot the expression of one selected marker for each cluster on top of the UMAP projection.<br>

For this we reload the annotated file as to not have to rerun all the above cells when reanalysing the data.

Further, I am performing a rudimentary **Differential Gene Expression Analysis** using the `sc.tl.rank_genes_groups()` method that determines differentially expressed genes between defined groups and ranks them by giving each gene a differential expression score.

In [ ]:
# Loading previously annotated data
adata = sc.read_h5ad(data_folder + r"CBA_SGNAtlas_Annotated.h5ad")

# And reducing the dataset to only SGNs
adata_SGNonly = adata[adata.obs["manual_celltypes"].isin(["Spiral Ganglion Neuron Type II",
                                                          "Spiral Ganglion Neuron Type Ia",
                                                          "Spiral Ganglion Neuron Type Ib",
                                                          "Spiral Ganglion Neuron Type Ic"])].copy()

In [ ]:
markers_to_plot = ["Gata3", # Type II
                   "Calb2", # Type Ia
                   "Ttn",   # Type Ib
                   "Lypd1"] # Type Ic

# Plotting
fig, axes = plt.subplots(2,3, figsize=(fig_width*3.4, fig_width*1.6), layout= "compressed")

dot_size = 50
line_width = 0.3
vmax = "p99"
sc.pl.umap(adata_SGNonly,
           color=["manual_celltypes"],
           edgecolor= "black",
           linewidth= line_width,
           size= dot_size,
           palette= my_cmap.colors[14:18],
           ax= axes[0, 0],
           show= False,
           vmax= vmax,
           title= "")

sc.pl.umap(adata_SGNonly[adata_SGNonly.obs["manual_celltypes"].isin(["Spiral Ganglion Neuron Type Ib", "Spiral Ganglion Neuron Type Ic"])],
           color=["age"],
           edgecolor= "black",
           linewidth= line_width,
           size= dot_size*2.3,
           palette= my_cmap.colors[4:],
           ax= axes[1, 0],
           show= False,
           vmax= vmax,
           title= "Age Difference")

for i, mark in enumerate(markers_to_plot):
    sc.pl.umap(adata_SGNonly,
               color= mark,
               size= dot_size,
               vmax= vmax,
               legend_loc= "none",
               edgecolor= "black",
               linewidth= line_width,
               ax= axes[int(i/2), (i%2)+1],
               title= mark,
               show= False)
    axes[int(i/2), (i%2)+1].set_ylabel("Normalized Expression")

fig.savefig(figures_folder + r"SGN_MarkerExpression_SGNplates.png")
plt.show()

Differential Gene Expression between age groups. Here we calculate the gene ranks for only Type Ic between age groups or only Type Ib between age groups.

In [ ]:
# Which cell type to analyse
cell_type = "Type Ic"

# slicing the data frame to contain only that one cell type
adata_DEG = adata_SGNonly[adata_SGNonly.obs["manual_celltypes"].str.contains(cell_type)].copy()

# Ranking differential gene groups in between ages (we compare 24mo to 3mo)
sc.tl.rank_genes_groups(adata_DEG, "age",
                        group= ["24 months"],
                        reference= "3 months",
                        method='wilcoxon',
                        n_genes= 10)

# Plotting
fig, axs = plt.subplots(1,2, figsize= (fig_width*2.4, fig_width*1.1), layout= "compressed", width_ratios= (1,2))

sc.pl.rank_genes_groups(adata_DEG, n_genes=10, ax= axs[1], show= False,
                        title= "DEGs 24 months vs. 3 months")
axs[1].set_axis_off()
axs[1].set_ylim(2,10)

sc.pl.rank_genes_groups_violin(adata_DEG, groups= ["24 months"],
                               gene_names= ["Prl", "Gh"],
                               ax= axs[0],
                               size= 2.5,
                               jitter= 0.25)
axs[0].set_title(f"SGN {cell_type}")
axs[0].set_zorder(3)

fig.savefig(figures_folder + f"DEGs_{cell_type}.png")
plt.show()

### Gene Expression Comparison

As supporting data to my cell stains I compare the expressions of the stained markers between young and old.

In [ ]:
# Creating a dataframe with the plotting data
data = dict()
adata = sc.read_h5ad(data_folder + r"CBA_SGNAtlas_Annotated.h5ad")

# For this comparison we use the median + log1p normalized counts because they start at 0
adata.X = adata.layers["log1p_normalized_counts"].copy()

# Filtering out only Type Ic nuclei
# Change this to Type Ia and Ib (list format)
adata_SGNonly = adata[adata.obs["manual_celltypes"].isin(["Spiral Ganglion Neuron Type Ic"])].copy()

# Extracting normalized gene counts for each of the transcripts
data["3 months"] = {"Lypd1": np.array(adata_SGNonly[adata_SGNonly.obs["age"] == "3 months", "Lypd1"].to_df()["Lypd1"].tolist()),
                    "Calb2": np.array(adata_SGNonly[adata_SGNonly.obs["age"] == "3 months", "Calb2"].to_df()["Calb2"].tolist())}
data["24 months"] = {"Lypd1": np.array(adata_SGNonly[adata_SGNonly.obs["age"] == "24 months", "Lypd1"].to_df()["Lypd1"].tolist()),
                     "Calb2": np.array(adata_SGNonly[adata_SGNonly.obs["age"] == "24 months", "Calb2"].to_df()["Calb2"].tolist())}


# Plotting
fig, axs = plt.subplots(1,2, figsize= (fig_width*1.6, fig_width*1), layout= "tight", sharey= True)

for i, marker in enumerate(["Lypd1", "Calb2"]):
    for j, age in enumerate(data.keys()):
        # making a bar plot
        axs[i].bar([age],
                np.mean(data[age][marker]),
                color= my_cmap.colors[j+4],
                    edgecolor= "black",
                    linewidth = 1,
                    yerr= np.std(data[age][marker]),
                    error_kw= {"ecolor": "black",
                                "linewidth": 1.5,
                                "capsize": 10},
                    zorder= 2)
        
        # Adding jitter points
        axs[i].scatter(np.random.normal(j, 0.1, len(data[age][marker])).tolist(),
           data[age][marker],
           color = "black",
           s = 6,
           zorder= 3)
        axs[i].set_title(marker)
    else:
        # We calculate a wilcoxon Rank sum test to check whether the means are actually not different
        print(f"{marker} Ranksum Test:\n", st.ranksums(data["3 months"][marker], data["24 months"][marker]))

axs[0].set_ylabel("Normalized Expression")
axs[0].set_ylim(-0.2, 4)

fig.savefig(figures_folder + r"Expression_MarkerGenes_Staining_SGNAtlas.png", dpi= 600)
plt.show()

### END OF SCRIPT
This notebook was created by *Johann Korn* on 01-10-2025.